<a href="https://colab.research.google.com/github/Sandeep-Rao-Sirikonda1/gitpract/blob/main/mnist_ann_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
pip install  gradio

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import gradio as gr
import numpy as np
from PIL import Image

# Define the neural network model
class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.layer1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(128, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = self.layer1(x)
        act1 = self.relu(out)
        act2 = self.layer2(act1)
        return act2

# Load the dataset
transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True,
transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
batch_size=64, shuffle=True)

# Initialize the model, loss function and optimizer
model = MNISTModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.005)

# Train the model
for epoch in range(10):
    for images, labels in train_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Define the Gradio interface function
def predict(image):


    # Access the image data from the 'layers' key (first layer)
    if 'layers' in image and image['layers']:  # Check if 'layers' key existsand is not empty
        image_np = image['layers'][0]  # Access the first layer
    else:
        raise ValueError("Input image data not found in expected format")

    # Convert to a NumPy array
    image_np = np.array(image_np)
    # Convert to PIL Image
    image_pil = Image.fromarray(image_np).convert("L")

    # Resize, to_tensor, and normalize the image
    image = transforms.functional.resize(image_pil, (28, 28))
    image = transforms.functional.to_tensor(image)
    image = transforms.functional.normalize(image, mean=(0.1307,),
std=(0.3081,))

    # Reshape image to (1, 784)
    image = image.unsqueeze(0).view(1, -1)

    # Perform inference
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)

    return predicted.item()

# Create Gradio interface
interface = gr.Interface(fn=predict, inputs="sketchpad", outputs="label",
live=True, description="Draw a digit and the model will predict it.")

# Launch the Gradio app
if __name__ == "__main__":
    interface.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d1705bab47217f29a6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
